In [1]:
import pandas as pd
from datetime import datetime
from pytz import timezone

In [2]:
raw_accidents_path = "./oak_data/raw_accidents.csv"
accidents_path = "./oak_data/accidents.csv"

In [3]:
df = pd.read_csv(raw_accidents_path, dtype={
    "Time Occurred": str
})

In [4]:
tz = timezone('US/Pacific')

def string_to_unix(row):
    d, t = row['Date Occurred'], row['Time Occurred']
    s = "{} {}".format(d, t)
    return tz.localize(datetime.strptime(s, '%m/%d/%Y %H%M'))

In [5]:
df['occured'] = df.apply(string_to_unix, axis=1)

In [6]:
df = df[df['occured'].apply(lambda d: d.year == 2019)]

In [7]:
def extract_location(location):
    s = location.strip()
    assert s[0] == '('
    assert s[-1] == ')'
    assert s.find(',') > -1
    lat = float(s[1:s.find(',')].strip())
    lon = float(s[s.find(',')+1:-1].strip())
    return pd.Series((lat, lon))

In [8]:
df[['latitude', 'longitude']] = df['Location'].apply(extract_location)

In [9]:
df = df[(df['latitude'] != 0) & (df['longitude'] != 0)]

In [10]:
df.to_csv(accidents_path)